# Простой RAG для проблем GitHub с использованием Hugging Face Zephyr и LangChain

_Автор: [Мария Халусова](https://github.com/MKhalusova)_

Этот блокнот демонстрирует, как можно быстро создать RAG (Retrieval Augmented Generation) для проблем (issues) проекта на GitHub, используя модель [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), и LangChain.


**Что такое RAG?**

RAG - это популярный подход к решению проблемы, когда мощная LLM не знает о конкретном контенте из-за того, что его нет в ее обучающих данных, или галлюцинирует, даже если видела его ранее. Такой специфический контент может быть проприетарным, конфиденциальным или, как в данном примере, недавно появившимся и часто обновляемым.

Если ваши данные статичны и не меняются регулярно, вы можете рассмотреть возможность дообучения большой модели. Однако во многих случаях дообучение может быть дорогостоящим, а при многократном повторении (например, для устранения дрейфа данных (address data drift) приводит к "сдвигу модели (model shift)". Это когда поведение модели изменяется нежелательным образом.

**RAG (Retrieval Augmented Generation, генерация с расширенным извлечением информации)** не требует дообучения модели. Вместо этого RAG работает, предоставляя LLM дополнительный контекст, который извлекается из соответствующих данных, чтобы она могла генерировать более обоснованный ответ.



Вот небольшая иллюстрация:

![RAG диаграмма](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/rag-diagram.png)

* Внешние данные преобразуются в векторы эмеддингов с помощью отдельной модели эмеддингов, и векторы хранятся в базе данных. Модели эмеддингов обычно имеют небольшой размер, поэтому регулярное обновление векторов эмеддингов быстрее, дешевле и проще, чем дообучение модели.

* В то же время отсутствие необходимости дообучения дает вам возможность поменять вашу LLM на более мощную, когда она станет доступной, или перейти на более компактную версию, если вам понадобится более быстрый инференс.

Давайте проиллюстрируем создание RAG с использованием открытой LLM, модели эмбеддингов и LangChain.

Сначала установите необходимые зависимости:

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [2]:
# Если вы используете Google Colab, вам может понадобиться запустить эту ячейку, чтобы убедиться, что вы используете UTF-8 для установки LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain

## Подготовка данных


В этом примере мы загрузим все проблемы (issues) (как открытые, так и закрытые) из [репозитория библиотеки PEFT](https://github.com/huggingface/peft).

Во-первых, вам необходимо получить [персональный токен доступа GitHub](https://github.com/settings/tokens?type=beta) чтобы получить доступ к API GitHub.

In [ ]:
from getpass import getpass
ACCESS_TOKEN = getpass("YOUR_GITHUB_PERSONAL_TOKEN")

Далее мы загрузим все проблемы (issues) в репозитория [huggingface/peft](https://github.com/huggingface/peft):
- По умолчанию предложения об изменении кода (pull requests) также считаются проблемами, но здесь мы решили исключить их из данных, установив `include_prs=False`.
- Установка `state = "all"` означает, что мы будем загружать как открытые, так и закрытые проблемы (issues).

In [5]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(
    repo="huggingface/peft",
    access_token=ACCESS_TOKEN,
    include_prs=False,
    state="all"
)

docs = loader.load()

Содержание отдельных проблем (issues) на GitHub может быть длиннее, чем то, что модель эмбеддингов может принять в качестве входных данных. Если мы хотим внедрить весь доступный контент, нам нужно разбить документы на фрагменты (chunk) соответствующего размера.

Наиболее распространенный и простой подход к фрагментации заключается в определении фиксированного размера фрагментов (chunk) и того, должно ли между ними быть какое-либо перекрытие. Сохранение некоторого перекрытия между фрагментами позволяет нам сохранить некоторый семантический контекст между фрагментами. Рекомендуемый сплиттер для общего текста - [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), и именно его мы будем здесь использовать. 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

## Создание эмбеддингов + retriever

Теперь, когда все документы имеют подходящий размер, мы можем создать базу данных с их эмбеддингами.

Для создания фрагментов эмбеддингов документов мы будем использовать модель эмбеддингов `HuggingFaceEmbeddings` и [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5). Есть много других моделей эмбеддингов, доступных на Hub, и вы можете отслеживать самые эффективные из них, проверяя [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).


Для создания базы векторов мы воспользуемся библиотекой `FAISS`, разработанной Facebook AI. Эта библиотека обеспечивает эффективный поиск сходства и кластеризацию плотных векторов (dense vectors), что нам и нужно. В настоящее время FAISS является одной из наиболее используемых библиотек для поиска NN в массивных наборах данных.

Мы получим доступ к модели эмбеддингов и FAISS через LangChain API.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

Нам нужен способ возврата (retrieve) документов по неструктурированному запросу. Для этого мы воспользуемся методом `as_retriever`, используя `db` в качестве основы:
- `search_type="similarity"` означает, что мы хотим выполнить поиск по сходству (similarity) между запросом и документами
- `search_kwargs={'k': 4}` задает retriever возвращать 4 лучших результата.


In [8]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

Векторная база данных и retriever настроены, осталось настроить следующий элемент цепочки - модель.

## Загрузка квантованной модели

Для этого примера мы выбрали [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), небольшую, но мощную модель.

Поскольку каждую неделю выходит множество моделей, вы можете захотеть заменить эту модель на самую последнюю и лучшую. Лучший способ отслеживать LLM с открытым исходным кодом - проверять [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

Чтобы ускорить инференс, мы загрузим квантованную версию модели:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Создание цепочки LLM

Наконец, у нас есть все, что нужно для создания цепочки LLM.

Во-первых, создадим конвейер генерации текста (text_generation pipeline), используя загруженную модель и ее токенизатор.

Затем создайте шаблон подсказки (prompt)  - он должен соответствовать формату модели, поэтому, если вы заменяете контрольную точку модели, убедитесь, что используете соответствующее форматирование.

In [15]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Примечание: _Вы также можете использовать `tokenizer.apply_chat_template` для преобразования списка сообщений (в виде dicts: `{'role': 'user', 'content': '(...)'}`) в строку с соответствующим форматом чата._


Наконец, нам нужно объединить `llm_chain` с retriever, чтобы создать RAG-цепочку. На последнем этапе генерации мы передаем оригинальный вопрос, а также извлеченные контекстные документы:

In [17]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


## Сравним результаты

Давайте посмотрим, как RAG влияет на генерирование ответов на специфические для библиотеки вопросы.

In [18]:
question = "How do you combine multiple adapters?"

Сначала посмотрим, какой ответ мы можем получить, используя только саму модель, без добавления контекста:

In [20]:
llm_chain.invoke({"context":"", "question": question})

" To combine multiple adapters, you need to ensure that they are compatible with each other and the devices you want to connect. Here's how you can do it:\n\n1. Identify the adapters you need: Determine which adapters you require to connect the devices you want to use together. For example, if you want to connect a USB-C device to an HDMI monitor, you may need a USB-C to HDMI adapter and a USB-C to USB-A adapter (if your computer only has USB-A ports).\n\n2. Connect the first adapter: Plug in the first adapter into the device you want to connect. For instance, if you're connecting a USB-C laptop to an HDMI monitor, plug the USB-C to HDMI adapter into the laptop's USB-C port.\n\n3. Connect the second adapter: Next, connect the second adapter to the first one. In this case, connect the USB-C to USB-A adapter to the USB-C port of the USB-C to HDMI adapter.\n\n4. Connect the final device: Finally, connect the device you want to use to the second adapter. For example, connect the HDMI cable

Как видите, модель интерпретировала вопрос как вопрос о физических компьютерных адаптерах, тогда как в контексте PEFT под "адаптерами" подразумеваются адаптеры LoRA.
Посмотрим, поможет ли добавление контекста из проблем (issues) GitHub дать более релевантный ответ:

In [21]:
rag_chain.invoke(question)

" Based on the provided context, it seems that combining multiple adapters is still an open question in the community. Here are some possibilities:\n\n  1. Save the output from the base model and pass it to each adapter separately, as described in the first context snippet. This allows you to run multiple adapters simultaneously and reuse the output from the base model. However, this approach requires loading and running each adapter separately.\n\n  2. Export everything into a single PyTorch model, as suggested in the second context snippet. This would involve saving all the adapters and their weights into a single model, potentially making it larger and more complex. The advantage of this approach is that it would allow you to run all the adapters simultaneously without having to load and run them separately.\n\n  3. Merge multiple Lora adapters, as mentioned in the third context snippet. This involves adding multiple distinct, independent behaviors to a base model by merging multipl

Как мы видим, дополнительный контекст помогает точно такой же модели дать гораздо более релевантный и обоснованный ответ на специфический для конкретной библиотеки вопрос.

Примечательно, что объединение нескольких адаптеров для инференса было добавлено в библиотеку, и эту информацию можно найти в документации, так что для следующей итерации этого RAG, возможно, стоит включить эмбеддинги документации.